# Project 2 Report
By Michael Geiger

### Project Overview
{Insert section here}

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import kaggle
import codecs
import statsmodels.formula.api as smf
from sklearn.linear_model import Lasso
from kaggle.api.kaggle_api_extended import KaggleApi
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
sns.set_context('talk')
sns.set_style('darkgrid')
sns.set_palette('colorblind')

In [5]:
url_teams = 'https://www.kaggle.com/tobycrabtree/nfl-scores-and-betting-data/downloads/nfl_teams.csv/11'
url_stadiums = 'https://www.kaggle.com/tobycrabtree/nfl-scores-and-betting-data/downloads/nfl_stadiums.csv/11'
url_scores = 'https://www.kaggle.com/tobycrabtree/nfl-scores-and-betting-data/downloads/spreadspoke_scores.csv/11'

In [6]:
#api = KaggleApi()
#nfl_teams = api.datasets_download_file("tobycrabtree/nfl-scores-and-betting-data","downloads/nfl_teams.csv/11", "nfl_teams.csv")

In [158]:
nfl_teams = pd.read_csv("nfl_teams.csv")
current_nfl_teams = pd.read_csv("https://raw.githubusercontent.com/CriticalPathTraining/PowerBiPartyPack/master/NFL/NFL%20Teams.csv")
nfl_stadiums = pd.read_csv("nfl_stadiums.csv", encoding='cp437')
nfl_scores = pd.read_csv("spreadspoke_scores.csv")
nfl_teams.head()

,team_name,team_name_short,team_id,team_id_pfr,team_conference,team_division,team_conference_pre2002,team_division_pre2002
0,Arizona Cardinals,Cardinals,ARI,CRD,NFC,NFC West,NFC,NFC West
1,Phoenix Cardinals,Cardinals,ARI,CRD,NFC,NaN,NFC,NFC East
2,St. Louis Cardinals,Cardinals,ARI,ARI,NFC,NaN,NFC,NFC East
3,Atlanta Falcons,Falcons,ATL,ATL,NFC,NFC South,NFC,NFC West
4,Baltimore Ravens,Ravens,BAL,RAV,AFC,AFC North,AFC,AFC Central


In [162]:
#print(nfl_teams)
#print(current_nfl_teams)
#current_nfl_teams.isnull().sum(axis=1)
cleaned_teams = nfl_teams.drop(labels=[1,2,15,20,22,26,30,37,38]).drop(labels=['team_id_pfr','team_conference_pre2002','team_division_pre2002'],axis=1).reset_index(drop=True)

In [165]:
cleaned_teams.columns=['Full Team', 'Team', 'ID', 'Conference', 'Division']

In [167]:
cleaned_teams

,Full Team,Team,ID,Conference,Division
0,Arizona Cardinals,Cardinals,ARI,NFC,NFC West
1,Atlanta Falcons,Falcons,ATL,NFC,NFC South
2,Baltimore Ravens,Ravens,BAL,AFC,AFC North
3,Buffalo Bills,Bills,BUF,AFC,AFC East
4,Carolina Panthers,Panthers,CAR,NFC,NFC South
5,Chicago Bears,Bears,CHI,NFC,NFC North
6,Cincinnati Bengals,Bengals,CIN,AFC,AFC North
7,Cleveland Browns,Browns,CLE,AFC,AFC North
8,Dallas Cowboys,Cowboys,DAL,NFC,NFC East
9,Denver Broncos,Broncos,DEN,AFC,AFC West


In [189]:
current_nfl_teams

,Team,Division,Conference,City,Team Roster,Team Logo
0,Bills,AFC-East,AFC,"Buffalo, NY",http://www.buffalobills.com/team/roster.html,https://cpt.azureedge.net/TeamLogoImages/bills...
1,Dolphins,AFC-East,AFC,"Miami, FL",http://www.miamidolphins.com/team/player-roste...,https://cpt.azureedge.net/TeamLogoImages/dolph...
2,Jets,AFC-East,AFC,"East Rutherford, NJ",http://www.newyorkjets.com/team/roster.html,https://cpt.azureedge.net/TeamLogoImages/jets-...
3,Patriots,AFC-East,AFC,"Foxboro, MA",http://www.patriots.com/team/roster,https://cpt.azureedge.net/TeamLogoImages/patri...
4,Bengals,AFC-North,AFC,"Cincinnati, OH",http://www.bengals.com/team/roster.html,https://cpt.azureedge.net/TeamLogoImages/benga...
5,Browns,AFC-North,AFC,"Cleveland, OH",http://www.clevelandbrowns.com/team/roster.html,https://cpt.azureedge.net/TeamLogoImages/brown...
6,Ravens,AFC-North,AFC,"Baltimore, MD",http://www.baltimoreravens.com/team/roster.html,https://cpt.azureedge.net/TeamLogoImages/raven...
7,Steelers,AFC-North,AFC,"Pittsburg, PA",http://www.steelers.com/team/roster.html,https://cpt.azureedge.net/TeamLogoImages/steel...
8,Colts,AFC-South,AFC,"Indianapolis, IN",http://www.colts.com/team/roster.html,https://cpt.azureedge.net/TeamLogoImages/colts...
9,Jaguars,AFC-South,AFC,"Jacksonville, FK",http://www.jaguars.com/team/roster.html,https://cpt.azureedge.net/TeamLogoImages/jagua...


In [191]:
cleaned_team_info = cleaned_teams.merge(current_nfl_teams, how='inner', on='Team', suffixes=('','_y'))
cleaned_team_info.drop(labels=['Division_y','Conference_y','Team Roster','Team Logo'], axis=1)

,Full Team,Team,ID,Conference,Division,City
0,Arizona Cardinals,Cardinals,ARI,NFC,NFC West,"Glendale, AZ"
1,Atlanta Falcons,Falcons,ATL,NFC,NFC South,"Atlanta, GA"
2,Baltimore Ravens,Ravens,BAL,AFC,AFC North,"Baltimore, MD"
3,Buffalo Bills,Bills,BUF,AFC,AFC East,"Buffalo, NY"
4,Carolina Panthers,Panthers,CAR,NFC,NFC South,"Charlotte, NC"
5,Chicago Bears,Bears,CHI,NFC,NFC North,"Chicago, IL"
6,Cincinnati Bengals,Bengals,CIN,AFC,AFC North,"Cincinnati, OH"
7,Cleveland Browns,Browns,CLE,AFC,AFC North,"Cleveland, OH"
8,Dallas Cowboys,Cowboys,DAL,NFC,NFC East,"Dallas, TX"
9,Denver Broncos,Broncos,DEN,AFC,AFC West,"Denver, CO"


In [194]:
nfl_stadiums.head(10)

,stadium_name,stadium_location,stadium_open,stadium_close,stadium_type,stadium_address,stadium_weather_station_code,stadium_weather_type,stadium_capacity,stadium_surface,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION
0,Alamo Dome,"San Antonio, TX",NaN,NaN,indoor,"100 Montana St, San Antonio, TX 78203",78203,dome,72000,FieldTurf,NaN,NaN,NaN,NaN,NaN
1,Alltel Stadium,"Jacksonville, FL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alumni Stadium,"Chestnut Hill, MA",NaN,NaN,outdoor,"Perimeter Rd, Chestnut Hill, MA 02467",02467,cold,NaN,Grass,NaN,NaN,NaN,NaN,NaN
3,Anaheim Stadium,"Anaheim, CA",1980.0,1994.0,outdoor,"2000 E Gene Autry Way, Anaheim, CA 92806",92806,warm,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arrowhead Stadium,"Kansas City, MO",1972.0,NaN,outdoor,"1 Arrowhead Dr, Kansas City, MO 64129",64129,cold,"76,416",Grass,US1MOJC0028,"KANSAS CITY 5.1 SE, MO US",39.06920,-94.48710,264.9
5,AT&T Stadium,"Arlington, TX",2009.0,NaN,retractable,"1 AT&T Way, Arlington, TX 76011",76011,dome,"80,000",FieldTurf,USC00410337,"ARLINGTON SIX FLAGS, TX US",32.75720,-97.07360,163.4
6,Atlanta-Fulton County Stadium,"Atlanta, GA",1966.0,1991.0,outdoor,"521 Capitol Avenue SE, Atlanta, Georgia 30312",30312,warm,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Balboa Stadium,"San Diego, CA",NaN,NaN,outdoor,"Balboa Stadium, San Diego, CA 92101",92101,warm,NaN,Grass,NaN,NaN,NaN,NaN,NaN
8,Bank of America Stadium,"Charlotte, NC",1996.0,2016.0,outdoor,"800 S Mint St, Charlotte, NC 28202",28202,moderate,"73,778",Grass,USW00013881,"CHARLOTTE DOUGLAS AIRPORT, NC US",35.22360,-80.95520,221.9
9,Busch Memorial Stadium,"St. Louis, MO",1966.0,1995.0,outdoor,"700 Clark Ave, St. Louis, MO 63102",63102,cold,NaN,NaN,USC00238791,"WEBSTER GROVES, MO US",38.56667,-90.36667,189.0


In [204]:
nfl_scores.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,09/02/1966,1966,1,False,Miami Dolphins,14,23,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71,NaN
1,09/03/1966,1966,1,False,Houston Oilers,45,7,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70,NaN
2,09/04/1966,1966,1,False,San Diego Chargers,27,7,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82,NaN
3,09/09/1966,1966,2,False,Miami Dolphins,14,19,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78,NaN
4,09/10/1966,1966,1,False,Green Bay Packers,24,3,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62,NaN


In [263]:
recent_regscores = nfl_scores.loc[lambda nfl_scores: nfl_scores['schedule_season'] >= 2010].loc[lambda nfl_scores: nfl_scores['schedule_playoff'] == False]
recent_regscores = recent_regscores.drop(labels=['schedule_playoff'], axis=1)
recent_regscores.head(10)

,schedule_date,schedule_season,schedule_week,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
10008,09/09/2010,2010,1,New Orleans Saints,14,9,Minnesota Vikings,NO,-5.0,49.5,Louisiana Superdome,False,72.0,0.0,NaN,DOME
10009,09/12/2010,2010,1,Buffalo Bills,10,15,Miami Dolphins,MIA,-3.0,39,Ralph Wilson Stadium,False,64.0,7.0,81,NaN
10010,09/12/2010,2010,1,Chicago Bears,19,14,Detroit Lions,CHI,-6.5,45,Soldier Field,False,75.0,1.0,45,NaN
10011,09/12/2010,2010,1,Houston Texans,34,24,Indianapolis Colts,IND,-1.0,48,Reliant Stadium,False,89.0,5.0,NaN,DOME (Open Roof)
10012,09/12/2010,2010,1,Jacksonville Jaguars,24,17,Denver Broncos,JAX,-3.0,41.5,EverBank Field,False,91.0,1.0,67,NaN
10013,09/12/2010,2010,1,New England Patriots,38,24,Cincinnati Bengals,NE,-5.5,45,Gillette Stadium,False,63.0,10.0,NaN,NaN
10014,09/12/2010,2010,1,New York Giants,31,18,Carolina Panthers,NYG,-6.0,41,MetLife Stadium,False,65.0,1.0,67,Rain
10015,09/12/2010,2010,1,Philadelphia Eagles,20,27,Green Bay Packers,GB,-3.0,48,Lincoln Financial Field,False,69.0,4.0,72,NaN
10016,09/12/2010,2010,1,Pittsburgh Steelers,15,9,Atlanta Falcons,ATL,-1.5,39.5,Heinz Field,False,65.0,9.0,67,NaN
10017,09/12/2010,2010,1,Seattle Seahawks,31,6,San Francisco 49ers,SF,-3.0,37,CenturyLink Field,False,64.0,11.0,NaN,NaN


In [258]:
recent_regscores.isnull().sum()

schedule_date             0
schedule_season           0
schedule_week             0
schedule_playoff          0
team_home                 0
score_home                0
score_away                0
team_away                 0
team_favorite_id          0
spread_favorite           0
over_under_line           0
stadium                   0
stadium_neutral           0
weather_temperature       0
weather_wind_mph          0
weather_humidity       1633
weather_detail         1542
dtype: int64

In [285]:
recent_regscores['stadium_neutral'].value_counts()

False    2276
True       28
Name: stadium_neutral, dtype: int64